In [39]:
import os, sys
import numpy as np
from tqdm import tqdm



In [34]:
## Define seed.
# try: seed = int(sys.argv[1])
seed = 654678 #47404
np.random.seed(seed)

In [35]:


#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Define trial types.
robots = [0,1,2,3]    # GW, NGW, GAL, NGAL

## Define number of presentations.
seqlen = [8,10,12]

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Generate trial order.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Define useful functions.
def max_count(arr):
    return np.unique(arr, return_counts=True)[-1].max()

## Define metadata.
ncol = len(robots)
nrow = len(seqlen)
colsum = sum(seqlen)

## Initialize trial info.
B, A = [arr.flatten() for arr in np.meshgrid(seqlen, robots)]
C = np.arange(A.size)

order = np.arange(A.size)
for _ in tqdm(range(int(1e3))):
    
    ## Permutate order.
    np.random.shuffle(order)
    A = A[order]; B = B[order]; C = C[order]
      
    ## Check #1: All rows have no more than two of the same robot type.
    counts = np.apply_along_axis(max_count, 1, A.reshape(nrow,ncol))
    if np.any(counts >= 3): continue
        
    ## Check #2: All rows have diversity of valence.
    valence = A.reshape(nrow,ncol) // 2
    if np.any(np.all(valence == 1, axis=1) | np.all(valence == 0, axis=1)): continue
        
    ## Check #3: All rows have diversity of action.
    action = A.reshape(nrow,ncol) % 2
    if np.any(np.all(action == 1, axis=1) | np.all(action == 0, axis=1)): continue
        
    ## Check #4: All columns have equal sums.
    if not np.all(B.reshape(nrow,ncol).sum(axis=0) == colsum): continue
        
    ## Check #5: All rows have variable repetitions.
    counts = np.apply_along_axis(max_count, 1, B.reshape(nrow,ncol))
    if np.any(counts >= 3): continue
        
    break
    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Fill in trials.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Reshape info.
A = A.reshape(nrow,ncol)
B = B.reshape(nrow,ncol)
C = C.reshape(nrow,ncol)

## Preallocate space.
robots = np.zeros((colsum, ncol), dtype=int)
stimuli = np.zeros_like(robots)

## Fill in trials.
for k in range(ncol):
    robots[:,k] = np.concatenate([np.tile(a,b) for a, b in zip(A[:,k],B[:,k])])
    stimuli[:,k] = np.concatenate([np.tile(c,b) for c, b in zip(C[:,k],B[:,k])])
    
## Randomize trial order.
order = np.arange(ncol)
for i in range(colsum):
    
    ## Permute order.
    np.random.shuffle(order)
    robots[i,:] = robots[i,order]
    stimuli[i,:] = stimuli[i,order]
    
## Re-order stimuli identitites.
shape = stimuli.shape
stimuli = stimuli.flatten()

dd = {}
for i, x in enumerate(stimuli):
    if not x in dd: dd[x] = len(dd)
    stimuli[i] = dd[x]

stimuli = stimuli.reshape(shape)

  2%|▏         | 22/1000 [00:00<00:00, 1073.86it/s]


In [18]:


## Dump to file.
with open('pit_runsheet_%s.txt' %seed, 'w') as f:
    
    ## Write latent order.
    f.write('Latent Order\n')
    f.write(np.array2string(A, separator=',') + '\n\n')
    
    ## Write robots.
    f.write('Robots\n')
    f.write(np.array2string(robots, separator=',') + '\n\n')
    
    ## Write stimuli.
    f.write('Stimuli\n')
    f.write(np.array2string(stimuli, separator=',') + '\n\n')

In [49]:
# # robots_13= np.array([[1,2,3,0],
# #     [0,2,1,3],
# #     [2,3,1,0],
# #     [0,3,2,1],
# #     [3,2,0,1],
# #     [3,2,1,0],
# #     [1,0,2,3],
# #     [2,3,1,0],
# #     [3,2,3,2],
# #     [3,2,3,2],
# #     [1,3,2,3],
# #     [2,3,1,3],
# #     [1,0,3,2],
# #     [2,1,0,3],
# #     [2,3,0,1],
# #     [0,3,2,1],
# #     [2,0,1,3],
# #     [1,2,0,3],
# #     [0,2,0,1],
# #     [2,0,1,0],
# #     [0,1,0,1],
# #     [0,1,0,1],
# #     [2,0,3,1],
# #     [0,2,1,3],
# #     [1,0,2,3],
# #     [2,3,0,1],
# #     [0,2,3,1],
# #     [0,1,3,2],
# #     [1,2,3,0],
# #     [0,3,1,2]
# #   ])
# # stimuli_13= np.array([
# #     [ 0, 1, 2, 3],
# #     [ 3, 1, 0, 2],
# #     [ 1, 2, 0, 3],
# #     [ 3, 2, 1, 0],
# #     [ 2, 1, 3, 0],
# #     [ 2, 1, 0, 3],
# #     [ 0, 3, 4, 2],
# #     [ 1, 2, 0, 3],
# #     [ 2, 4, 0, 1],
# #     [ 3, 1, 4, 0],
# #     [ 2, 3, 1, 4],
# #     [ 4, 2, 3, 1],
# #     [ 1, 2, 3, 5],
# #     [ 3, 1, 5, 4],
# #     [ 4, 3, 6, 5],
# #     [ 4, 5, 7, 6],
# #     [ 7, 5, 4, 6],
# #     [ 4, 7, 6, 5],
# #     [ 6, 5, 7, 8],
# #     [ 7, 8, 5, 6],
# #     [ 9, 7, 6, 8],
# #     [ 8, 9, 7, 5],
# #     [ 9, 5, 8, 6],
# #     [ 5, 8, 7, 9],
# #     [ 8, 5, 9, 6],
# #     [ 9, 8, 5, 7],
# #     [ 6, 7, 8, 9],
# #     [ 8, 6, 7, 9],
# #     [ 8, 9, 6, 7],
# #     [ 9, 6, 7, 8]
# #   ])

# # robots_28=np.array([
# #     [1,1,0,2],
# #     [1,0,2,1],
# #     [2,0,1,1],
# #     [1,2,1,0],
# #     [0,2,1,1],
# #     [2,1,0,1],
# #     [2,0,1,1],
# #     [1,1,2,0],
# #     [3,1,0,2],
# #     [1,2,0,3],
# #     [1,2,0,3],
# #     [2,0,1,3],
# #     [2,3,1,3],
# #     [1,3,2,3],
# #     [3,3,2,1],
# #     [3,3,2,1],
# #     [1,3,2,3],
# #     [2,3,1,3],
# #     [3,2,1,3],
# #     [2,1,3,3],
# #     [3,0,2,1],
# #     [2,3,0,1],
# #     [3,0,0,2],
# #     [0,0,2,3],
# #     [3,0,0,2],
# #     [2,0,0,3],
# #     [2,0,0,3],
# #     [3,0,2,0],
# #     [0,3,0,2],
# #     [0,0,2,3]
# #   ])
# # stimuli_28 = np.array([
# #     [ 0, 1, 2, 3],
# #     [ 1, 2, 3, 0],
# #     [ 3, 2, 0, 1],
# #     [ 1, 3, 0, 2],
# #     [ 2, 3, 1, 0],
# #     [ 3, 1, 2, 0],
# #     [ 3, 2, 0, 1],
# #     [ 1, 0, 3, 2],
# #     [ 3, 0, 2, 1],
# #     [ 0, 3, 2, 4],
# #     [ 1, 4, 2, 3],
# #     [ 0, 3, 1, 4],
# #     [ 5, 1, 2, 4],
# #     [ 1, 3, 5, 4],
# #     [ 4, 3, 5, 2],
# #     [ 6, 7, 5, 4],
# #     [ 5, 7, 6, 4],
# #     [ 4, 5, 6, 7],
# #     [ 5, 6, 7, 8],
# #     [ 6, 5, 8, 7],
# #     [ 8, 7, 5, 9],
# #     [ 9, 8, 4, 6],
# #     [ 8, 9, 6, 5],
# #     [ 6, 9, 5, 8],
# #     [ 8, 6, 9, 5],
# #     [ 7, 9, 6, 8],
# #     [ 6, 9, 7, 8],
# #     [ 8, 7, 6, 9],
# #     [ 7, 8, 9, 6],
# #     [ 9, 7, 6, 8]
# #   ])


# robots_13=np.array([
#    [3,0,3,2],
# [2,0,3,3],
# [0,3,3,2],
# [2,3,0,3],
# [3,0,2,3],
# [3,0,3,2],
# [3,2,1,3],
# [0,1,3,2],
# [3,1,3,0],
# [2,0,1,3],
# [3,2,0,1],
# [1,3,2,0],
# [0,1,2,0],
# [2,0,0,1],
# [1,2,3,0],
# [1,0,2,3],
# [2,0,1,3],
# [1,2,3,0],
# [3,0,2,1],
# [2,1,0,3],
# [1,0,3,1],
# [1,1,3,0],
# [1,0,1,3],
# [0,1,2,1],
# [1,0,1,3],
# [1,1,0,2],
# [3,2,1,1],
# [1,3,2,1],
# [1,1,3,2],
# [1,3,2,1]
#   ])
  
# stimuli_13=np.array([
# [0,1,2,3],
# [3,1,0,2],
# [1,2,0,3],
# [3,2,1,0],
# [2,1,3,0],
# [2,1,0,3],
# [0,3,4,2],
# [1,4,0,3],
# [2,4,0,1],
# [3,1,4,0],
# [2,3,1,4],
# [4,2,3,1],
# [1,4,3,5],
# [3,1,5,4],
# [4,3,6,5],
# [4,5,7,6],
# [7,5,4,6],
# [4,7,6,5],
# [6,5,7,8],
# [7,8,5,6],
# [9,5,6,8],
# [8,9,6,5],
# [9,5,8,6],
# [5,8,7,9],
# [8,5,9,6],
# [9,8,5,7],
# [6,7,8,9],
# [8,6,7,9],
# [8,9,6,7],
# [9,6,7,8] 
      
#    ])


# robots_28= np.array([
#     [2,1,2,3],
# [1,2,3,2],
# [3,2,2,1],
# [1,3,2,2],
# [2,3,1,2],
# [3,1,2,2],
# [3,2,2,1],
# [1,2,3,2],
# [3,2,2,1],
# [2,3,2,0],
# [1,0,2,3],
# [1,3,1,0],
# [1,1,2,0],
# [1,3,1,0],
# [0,3,1,0],
# [3,3,1,0],
# [1,3,2,0],
# [0,1,2,3],
# [1,2,3,0],
# [2,1,0,3],
# [0,3,1,0],
# [0,0,0,2],
# [0,0,2,1],
# [2,0,1,0],
# [0,2,0,1],
# [3,0,2,0],
# [2,0,3,0],
# [0,3,2,0],
# [3,0,0,2],
# [0,3,2,0]
#   ])
# stimuli_28=np.array([
#     [0,1,2,3],
# [1,2,3,0],
# [3,2,0,1],
# [1,3,0,2],
# [2,3,1,0],
# [3,1,2,0],
# [3,2,0,1],
# [1,0,3,2],
# [3,0,2,1],
# [0,3,2,4],
# [1,4,2,3],
# [5,3,1,4],
# [5,1,2,4],
# [1,3,5,4],
# [4,3,5,4],
# [3,7,5,4],
# [5,7,6,4],
# [4,5,6,7],
# [5,6,7,8],
# [6,5,8,7],
# [8,7,5,9],
# [9,8,4,6],
# [8,9,6,5],
# [6,9,5,8],
# [8,6,9,5],
# [7,9,6,8],
# [6,9,7,8],
# [8,7,6,9],
# [7,8,9,6],
# [9,7,6,8]
#   ])

In [335]:
runsheet_s28 = { #noam sep 15 22
  'robots': [
[2,1,2,3],
[1,2,3,2],
[3,2,2,1],
[1,3,2,2],
[2,3,1,2],
[3,1,2,2],
[3,2,2,1],
[1,2,3,2],
[3,2,2,1],
[2,3,2,0],
[1,0,2,3],
[1,3,1,0],
[1,1,2,0],
[1,3,1,0],
[0,3,1,0],
[3,3,1,0],
[1,3,2,0],
[0,1,2,3],
[1,2,3,0],
[2,1,0,3],
[0,3,1,0],
[0,0,0,2],
[0,0,2,1],
[2,0,1,0],
[0,2,0,1],
[3,0,2,0],
[2,0,3,0],
[0,3,2,0],
[3,0,0,2],
[0,3,2,0]
  ],
  'stimuli': [
[0,1,2,3],
[1,2,3,0],
[3,2,0,1],
[1,3,0,2],
[2,3,1,0],
[3,1,2,0],
[3,2,0,1],
[1,0,3,2],
[3,0,2,1],
[0,3,2,4],
[1,4,2,3],
[5,3,1,4],
[5,1,2,4],
[1,3,5,4],
[4,3,5,4],
[3,7,5,4],
[5,7,6,4],
[4,5,6,7],
[5,6,7,8],
[6,5,8,7],
[8,7,5,9],
[9,8,4,6],
[8,9,6,5],
[6,9,5,8],
[8,6,9,5],
[7,9,6,8],
[6,9,7,8],
[8,7,6,9],
[7,8,9,6],
[9,7,6,8]
  ]
}

runsheet_s13 = { # noam 15 sep 22
  'robots': [
   [3,0,3,2],
[2,0,3,3],
[0,3,3,2],
[2,3,0,3],
[3,0,2,3],
[3,0,3,2],
[3,2,1,3],
[0,1,3,2],
[3,1,3,0],
[2,0,1,3],
[3,2,0,1],
[1,3,2,0],
[0,1,2,0],
[2,0,0,1],
[1,2,3,0],
[1,0,2,3],
[2,0,1,3],
[1,2,3,0],
[3,0,2,1],
[2,1,0,3],
[1,0,3,1],
[1,1,3,0],
[1,0,1,3],
[0,1,2,1],
[1,0,1,3],
[1,1,0,2],
[3,2,1,1],
[1,3,2,1],
[1,1,3,2],
[1,3,2,1]
  ],
  
  
  'stimuli': [
[0,1,2,3],
[3,1,0,2],
[1,2,0,3],
[3,2,1,0],
[2,1,3,0],
[2,1,0,3],
[0,3,4,2],
[1,4,0,3],
[2,4,0,1],
[3,1,4,0],
[2,3,1,4],
[4,2,3,1],
[1,4,3,5],
[3,1,5,4],
[4,3,6,5],
[4,5,7,6],
[7,5,4,6],
[4,7,6,5],
[6,5,7,8],
[7,8,5,6],
[9,5,6,8],
[8,9,6,5],
[9,5,8,6],
[5,8,7,9],
[8,5,9,6],
[9,8,5,7],
[6,7,8,9],
[8,6,7,9],
[8,9,6,7],
[9,6,7,8] 
      
   ]
}

stimuli_28 = np.array(runsheet_s28['stimuli'])
robots_28 = np.array(runsheet_s28['robots'])

stimuli_13 = np.array(runsheet_s13['stimuli'])
robots_13 =  np.array(runsheet_s13['robots'])

In [336]:
print(stimuli_28[(robots_28==0)])
print(stimuli_28[(robots_28==1)])
print(stimuli_28[(robots_28==2)])
print(stimuli_28[(robots_28==3)])


[4 4 4 4 4 4 4 4 4 4 8 8 8 9 9 8 4 8 9 9 8 8 9 9 8 9 8 8 9 8 9 9 8]
[1 1 1 1 1 1 1 1 1 1 5 1 5 1 1 5 5 5 5 5 5 5 5 5 5 5]
[0 2 2 0 2 0 0 2 2 0 2 0 2 0 0 2 0 2 0 2 2 2 6 6 6 6 6 6 6 6 6 6 6 6 6]
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 7 7 7 7 7 7 7 7 7 7 7]


In [337]:
print(stimuli_13[(robots_13==0)])
print(stimuli_13[(robots_13==1)])
print(stimuli_13[(robots_13==2)])
print(stimuli_13[(robots_13==3)])


[1 1 1 1 1 1 1 1 1 1 1 1 5 1 5 5 5 5 5 5 5 5 5 5 5 5 5]
[4 4 4 4 4 4 4 4 4 4 4 4 8 8 9 8 8 9 9 8 8 9 8 9 9 8 8 9 8 9 8 9 9 8]
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 7 7 7 7 7 7 7 7 7 7 7]
[0 2 0 2 2 0 2 0 2 0 2 0 0 2 0 2 0 0 2 2 6 6 6 6 6 6 6 6 6 6 6 6 6 6]


In [340]:
n_exposure_tmp = np.array([sum(stimuli_28.flatten()==i) for i in range(10)])
print(n_exposure_tmp)
print(n_exposure)

[10 13 12 15 11 13 13 11 12 10]
[10 13 10 14 12 14 14 11 12 10]


In [380]:
#new stim/ 13a

robots_13_a_tmp = np.flip(robots_13, 0)

robots_13_a = np.where(robots_13_a_tmp==0, 3, np.nan)
robots_13_a = np.where(robots_13_a_tmp==1, 2, robots_13_a)
robots_13_a = np.where(robots_13_a_tmp==2, 1, robots_13_a)
robots_13_a = np.where(robots_13_a_tmp==3, 0, robots_13_a)

In [381]:
stimuli_13_a = np.flip(stimuli_13, 0)

In [382]:
print(np.array([sum(stimuli_13_a.flatten()==i) for i in range(10)]))
print(np.array([sum(stimuli_13.flatten()==i) for i in range(10)]))

[10 13 10 14 12 14 14 11 12 10]
[10 13 10 14 12 14 14 11 12 10]


In [383]:
print(stimuli_13[(robots_13==0)], len(stimuli_13[(robots_13==0)]))
print(stimuli_13[(robots_13==1)], len(stimuli_13[(robots_13==1)]))
print(stimuli_13[(robots_13==2)], len(stimuli_13[(robots_13==2)]))
print(stimuli_13[(robots_13==3)], len(stimuli_13[(robots_13==3)]))


[1 1 1 1 1 1 1 1 1 1 1 1 5 1 5 5 5 5 5 5 5 5 5 5 5 5 5] 27
[4 4 4 4 4 4 4 4 4 4 4 4 8 8 9 8 8 9 9 8 8 9 8 9 9 8 8 9 8 9 8 9 9 8] 34
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 7 7 7 7 7 7 7 7 7 7 7] 25
[0 2 0 2 2 0 2 0 2 0 2 0 0 2 0 2 0 0 2 2 6 6 6 6 6 6 6 6 6 6 6 6 6 6] 34


In [384]:
print(stimuli_13_a[(robots_13_a==0)], len(stimuli_13_a[(robots_13_a==0)]))
print(stimuli_13_a[(robots_13_a==1)], len(stimuli_13_a[(robots_13_a==1)]))
print(stimuli_13_a[(robots_13_a==2)], len(stimuli_13_a[(robots_13_a==2)]))
print(stimuli_13_a[(robots_13_a==3)], len(stimuli_13_a[(robots_13_a==3)]))


[6 6 6 6 6 6 6 6 6 6 6 6 6 6 2 2 0 2 0 0 0 2 2 0 2 0 2 0 2 0 0 2 0 2] 34
[7 7 7 7 7 7 7 7 7 7 7 3 3 3 3 3 3 3 3 3 3 3 3 3 3] 25
[9 8 8 9 8 9 8 9 9 8 8 9 8 9 9 8 8 9 9 8 8 8 4 4 4 4 4 4 4 4 4 4 4 4] 34
[5 5 5 5 5 5 5 5 5 5 5 5 1 5 1 5 1 1 1 1 1 1 1 1 1 1 1] 27


In [385]:
n_exposure = np.array([sum(stimuli_13.flatten()==i) for i in range(10)])
n_sham = np.round(n_exposure*.2)

In [386]:
print(n_sham/n_exposure)

[0.2        0.23076923 0.2        0.21428571 0.16666667 0.21428571
 0.21428571 0.18181818 0.16666667 0.2       ]


In [399]:
print(n_exposure, np.sum(n_exposure))
print(n_sham, np.sum(n_sham))

[10 13 10 14 12 14 14 11 12 10] 120
[2. 3. 2. 3. 2. 3. 3. 2. 2. 2.] 24.0


In [388]:
# n_sham_13
n_sham_dict = {i: np.concatenate([np.zeros(int(i_n_sham)),np.ones(int(i_n_true-i_n_sham))])
                            for i,(i_n_sham,i_n_true) in enumerate(zip(n_sham, n_exposure ))}


for i in n_sham_dict:
    np.random.shuffle(n_sham_dict[i])


In [389]:
n_sham_dict_copy_13 = n_sham_dict.copy()
n_sham_dict_copy_13_a = n_sham_dict.copy()

In [390]:
sham_13_flat=stimuli_13.copy().flatten()
sham_13_a_flat=stimuli_13_a.copy().flatten()

In [391]:
for i, val in enumerate(stimuli_13.flatten()):

    #assign
    sham_13_flat[i] = n_sham_dict_copy_13[val][0]  
    #remove
    n_sham_dict_copy_13[val] = np.delete(n_sham_dict_copy_13[val] , 0) 

In [392]:
for i, val in enumerate(stimuli_13_a.flatten()):

    #assign
    sham_13_a_flat[i] = n_sham_dict_copy_13_a[val][0]  
    #remove
    n_sham_dict_copy_13_a[val] = np.delete(n_sham_dict_copy_13_a[val] , 0) 


In [393]:
sham_13 = sham_13_flat.reshape(30,4)
sham_13_a = sham_13_a_flat.reshape(30,4)

In [404]:
## Dump to file.
with open('pit_runsheet_13.txt', 'w') as f:
    
    # ## Write latent order.
    # f.write('Latent Order\n')
    # f.write(np.array2string(A, separator=',') + '\n\n')
    
    ## Write robots.
    f.write('robots\n')
    f.write(np.array2string(np.array(robots_13,dtype=int), separator=',') + '\n\n')
    
    ## Write stimuli.
    f.write('stimuli\n')
    f.write(np.array2string(stimuli_13, separator=',') + '\n\n')

     ## Write stimuli.
    f.write('sham\n')
    f.write(np.array2string(sham_13, separator=',') + '\n\n')

In [400]:
# print(n_exposure_13[1] + n_exposure_13[5], n_sham_13[1]+n_sham_13[5])
# print(n_exposure_13[4] + n_exposure_13[8]+ n_exposure_13[9], n_sham_13[4]+n_sham_13[8]+n_sham_13[9])
# print(n_exposure_13[3] + n_exposure_13[7], n_sham_13[3]+n_sham_13[7])
# print(n_exposure_13[0] + n_exposure_13[2]+ n_exposure_13[6], n_sham_13[0]+n_sham_13[2]+n_sham_13[6])
